In [1]:
import torch
from PIL import Image
import requests
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.nn.functional as F
import numpy as np
from skimage.measure import find_contours

from matplotlib import patches,  lines
from matplotlib.patches import Polygon

torch.set_grad_enabled(False);

In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [4]:
gqa_val_q=json.load(
    open(os.path.join(GQA_PATH,'original','val_balanced_questions.json')
         ,'r')
)
print (len(gqa_val_q))
names=list(gqa_val_q.keys())

val_graphs=json.load(
    open(os.path.join(GQA_PATH,'val_sceneGraphs.json')
         ,'r')
)
print (len(val_graphs))

132062
10696


In [5]:
import skimage
from skimage import io as skimage_io
from skimage import transform as skimage_transform

def get_tokens(text_queries):
    tokenized_queries = np.array([
        module.tokenize(q, config.dataset_configs.max_query_length)
        for q in text_queries
    ])
    # Pad tokenized queries to avoid recompilation if number of queries changes:
    tokenized_queries = np.pad(
        tokenized_queries,
        pad_width=((0, 100 - len(text_queries)), (0, 0)),
        constant_values=0)
    return tokenized_queries

def get_gqa_feat(img_id):
    # Load example image:
    filename = os.path.join(GQA_PATH,'images',img_id+'.jpg')
    image_uint8 = skimage_io.imread(filename)
    image = image_uint8.astype(np.float32) / 255.0

    # Pad to square with gray pixels on bottom and right:
    h, w, _ = image.shape
    size = max(h, w)
    image_padded = np.pad(
        image, ((0, size - h), (0, size - w), (0, 0)), constant_values=0.5)

    # Resize to model input size:
    input_image = skimage.transform.resize(
        image_padded,
        (840, 840),
        anti_aliasing=True)
    return input_image

In [6]:
import clip

CUDA_DEVICE=15
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.to(device)
model.eval();

clip_model, preprocess = clip.load("ViT-B/32",device=device)

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import inflect
stemmer = inflect.engine()

In [8]:
def plot_inference(im, caption, plot=True):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0).cuda()

    # propagate through the model
    memory_cache = model(img, [caption], encode_and_save=True)
    outputs = model(img, [caption], encode_and_save=False, memory_cache=memory_cache)
    #print (outputs['pred_logits'])

    # keep only predictions with 0.7+ confidence
    probas = 1 - outputs['pred_logits'].softmax(-1)[0, :, -1].cpu()
    keep = (probas > 0.7).cpu()

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'].cpu()[0, keep], im.size)

    # Extract the text spans predicted by each box
    positive_tokens = (outputs["pred_logits"].cpu()[0, keep].softmax(-1) > 0.1).nonzero().tolist()
    predicted_spans = defaultdict(str)
    for tok in positive_tokens:
        item, pos = tok
        if pos < 255:
            span = memory_cache["tokenized"].token_to_chars(0, pos)
            predicted_spans [item] += " " + caption[span.start:span.end]

    labels = [predicted_spans [k] for k in sorted(list(predicted_spans .keys()))]
    if plot:
        plot_results(im, probas[keep], bboxes_scaled, labels)
    return probas[keep], bboxes_scaled.tolist(), labels

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
        masks = [None for _ in range(len(scores))]
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
            continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=c)
            ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):
    #for tt in results.values():
    if True:
        bboxes = results['boxes']
        labels = results['labels']
        scores = results['scores']
        #keep = scores >= 0.0
        #bboxes = bboxes[keep].tolist()
        #labels = labels[keep].tolist()
        #scores = scores[keep].tolist()
    #print(torchvision.ops.box_iou(tt['boxes'].cpu().detach(), torch.as_tensor([[xmin, ymin, xmax, ymax]])))
    
    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

In [9]:
def vis_bbox_only(bbox,scores,input_image,gt_bbox=None,text=None,threshold=0.2,vis_pred=True):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    if vis_pred:
        for i,box in enumerate(bbox):
            score=scores[i]
            if score<threshold:
                continue
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
            ax.text(
                cx - w / 2,
                cy + h / 2 + 0.015,
                f'{score:1.2f}',
                ha='left',
                va='top',
                color='red',
                bbox={
                    'facecolor': 'white',
                    'edgecolor': 'red',
                    'boxstyle': 'square,pad=.3'
                })
    if text is not None:
        ax.set_title(text,  fontsize=12)
    if gt_bbox is not None:
        for i,box in enumerate(gt_bbox):
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'b')

In [10]:
import IPython.display as display

def size_aware_pred(bb):
    if bb[2]>0.3 or bb[3]>0.3:
        return 'large'
    else:
        return 'small'
def pos_aware_pred(valid_bbox,candidates):
    if 'bottom' in candidates:
        if valid_bbox[1]>0.5:
            return 'bottom'
        else:
            return 'top'
    elif 'right' in candidates:
        if valid_bbox[0]<0.5:
            return 'left'
        else:
            return 'right'

In [11]:
def transform_coord(bbox,img_id):
    width=val_graphs[img_id]['width']
    height=val_graphs[img_id]['height']
    size = max(height, width)
    x=bbox[0]
    y=bbox[1]
    w=bbox[2]-bbox[0]
    h=bbox[3]-bbox[1]
    #print (bbox)
    return [(x+w/2)/size,(y+h/2)/size,w/size,h/size]

In [12]:
def preprocess_mdert_result(scores,bboxs,labels,obj,img_id):
    sf_s=[]
    sf_bbox=[]
    labels=[''.join(label.split(' ')) for label in labels]
    flag=False
    if obj in labels:
        flag=True
    #print (obj,labels)
    for i,label in enumerate(labels):
        if flag:
            if obj==label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
        else:
            if obj in label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
    #print (len(sf_s))
    if len(sf_bbox)==0:
        for i,label in enumerate(labels):
            sf_s.append(scores[i])
            sf_bbox.append(bboxs[i])
    max_s=max(sf_s)
    max_id=sf_s.index(max_s)
    return_bbox=sf_bbox[max_id]
    coord=transform_coord(return_bbox,img_id)
    return max_s.item(),coord
#only consider one bbox now

In [13]:
def generate_phrase_ground(img_id,cap,obj):
    cap=cap+' '
    im=Image.open(os.path.join(GQA_PATH,'images',img_id+'.jpg'))
    try:
        scores,bboxs,labels=plot_inference(im, cap, plot=False)
    except:
        print ('Invalid caption generation for',img_id,obj,cap)
        scores=[torch.Tensor([0.99])]
        bboxs=[[0.5,0.5,0.99,0.99]]
        labels=[obj]
    if len(bboxs)==0:
        max_s=0.99
        valid_bbox=[0.5,0.5,0.99,0.99]
    else:
        # print (scores,bboxs)
        max_s,valid_bbox=preprocess_mdert_result(scores,bboxs,labels,obj,img_id)
    return max_s,valid_bbox

def bbox_generator(layout,idx,img_id,ques_id):
    if layout[idx]['operation']=='select':
        #print (idx)
        try:
            scenic_result=load_pkl(os.path.join(GQA_PATH,
                                                'meta_one-all-scenic',
                                                ques_id+'.pkl'))
            obj=layout[idx]['argument'][0]
            scores=[s['score'] for s in scenic_result[obj]]
            bboxs=[s['bbox'] for s in scenic_result[obj]]
        except:
            bboxs=[]
            obj=layout[idx]['argument'][0]
        if len(bboxs)>0:
            max_s=max(scores)
            max_idx=scores.index(max_s)
            bbox=bboxs[max_idx]
            cap=obj
        else:
            cap=obj
            split_obj=''.join(obj.split(' '))
            #print (cap,split_obj)
            max_s,bbox=generate_phrase_ground(img_id,cap,split_obj)
            #print (max_s,bbox)
    else:
        cap,obj=cap_generator(layout,idx,img_id)
        max_s,bbox=generate_phrase_ground(img_id,cap,obj)
    return max_s,bbox,cap        
            
def cap_generator(layout,idx,img_id):
    words=[]
    dep=layout[idx]['dependencies']
    word=layout[idx]['argument'][0]
    words.append(word)
    obj=''.join(word.split(' '))
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            cur_step=layout[dep[0]]
            dep=cur_step['dependencies']
            arg=cur_step['argument'][0]
            words.append(arg)
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap,obj

In [14]:
def cap_generator_query(layout,idx,img_id):
    words=['what']
    dep=layout[idx]['dependencies']
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            new_sent=[]
            while len(dep)>0:
                cur_step=layout[dep[0]]
                dep=cur_step['dependencies']
                arg=cur_step['argument'][0]
                new_sent.append(arg)
            words.append(' '.join(new_sent))
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap

In [15]:
def get_cropped(bb,input_image,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    if showing:
        display.display(trans_crop)
    return trans_crop

def clip_aware_pred(img_feat,valid_bbox,candidates,showing=False):
    #tokens=clip.tokenize(candidates)
    trans_crop=get_cropped(valid_bbox,img_feat)
    if showing:
        display.display(trans_crop)
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(candidates).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (indices[0][0].item())
    ans=candidates[indices[0][0].item()]
    return ans

In [16]:
def show_img_with_text(img_path,texts):
    f = Image.open(img_path)
    img = f.convert('RGB')
    img = np.asarray(img, dtype=np.float32)
    
    fig = plt.figure()
    ax=fig.add_subplot(1,1,1)
    ax.set_title(texts,  fontsize=12)
    ax.imshow(img.astype(np.uint8))  

In [17]:
position_holder=['left','right','bottom','top','front']

In [18]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [19]:
def get_nouns(ques):
    nouns=[]
    tokens_tag = pos_tag(ques.replace('?','').split(' '))
    for w,pos in tokens_tag:
        if pos in ['NN','NNS'] and w not in position_holder:
            nouns.append(lemmatizer.lemmatize(w))
    return nouns

def verify_noun(nouns,obj):
    if obj in nouns or \
    lemmatizer.lemmatize(obj) in nouns or\
    stemmer.singular_noun(obj) in nouns:
        return True
    elif len(obj.split(' '))>1:
        words=obj.split(' ')
        for w in words:
            #print (w)
            if lemmatizer.lemmatize(w) in nouns or\
            stemmer.singular_noun(w) in nouns:
                return True
    elif obj in ['he','she','they']:
        return True
    return False

In [20]:
heur_rela=['to the left of','to the right of',
           'above','under','on top of','below',
           'beneath','underneath']
#if color on the last argument, use phrase grounding and binary color function

In [21]:
scene_list=['store', 'city', 'street', 'cloudy', 'beach', 'cloudless', 
             'sunny', 'pavement', 'outdoors', 'restroom', 'ocean',
             'sidewalk', 'skate park', 'indoors', 'field', 'garden',
             'sea', 'park', 'kitchen', 'road', 'living room', 'garage', 
             'yard', 'foggy', 'clear', 'town', 'airport', 'zoo', 'restaurant',
             'porch', 'highway', 'lawn', 'shore', 'desert', 'pen', 'forest', 
             'stadium', 'meadow', 'lake', 'bathroom', 'bedroom', 'swimming pool',
             'walkway', 'display', 'mall', 'museum', 'path', 'partly cloudy', 
             'parking lot', 'supermarket', 'overcast', 'station', 'tunnel', 
             'runway', 'pasture', 'train station', 'church', 'courtyard', 
             'office', 'pub', 'backyard', 'library', 'marina', 'hallway', 
             'river', 'shop', 'plain', 'cafe', 'market', 'place', 'gas station', 
             'harbor', 'terminal', 'stage', 'hotel', 'roadway', 'cafeteria', 
             'stormy', 'rainy', 'hotel room', 'railroad', 'bakery', 'pond',
             'kiosk', 'dining room', 'farm', 'patio', 'hangar', 'auditorium', 
             'classroom', 'salon', 'school', 'attic', 'village', 'swamp']

color_list=['white', 'black', 'blue', 'brown', 'gray', 'green', 'red',
            'yellow', 'orange', 'pink', 'silver', 'dark', 'tan', 'purple', 
            'gold', 'blond', 'beige', 'light brown', 'light blue', 'dark brown',
            'maroon', 'cream colored', 'dark blue', 'khaki']

material_list=['wood', 'metal', 'plastic', 'glass', 'concrete', 'brick',
               'leather', 'porcelain']

In [22]:
def generate_rela_query(layout,img_id,ques_id,showing=False,texts=None):
    cap=cap_generator_query(layout,len(layout)-1,img_id)
    cap=cap.replace('what','item')
    score,bbox=generate_phrase_ground(img_id,cap,'item')
    input_image=get_gqa_feat(img_id)
    pred,_=ans_open_query_candi(bbox,input_image,ques_id,showing)
    if showing:
        texts+='\nCaption:'+cap
        texts+='\nPrediction:'+pred
        vis_bbox_only([bbox],[score],input_image,text=texts)
    return pred

In [23]:
valid_ans_cat=load_pkl('../gqa_t5_pred/ans_per_cat_fix.pkl')

In [24]:
for name in valid_ans_cat:
    print (name)
    print ('\t',valid_ans_cat[name])

person
	 ['surfer', 'spectator', 'officer', 'batter', 'catcher', 'girl', 'biker', 'man', 'boy', 'policeman', 'skater', 'skateboarder', 'player', 'passenger', 'guy', 'woman', 'mother', 'lady', 'jockey', 'toddler', 'person', 'snowboarder', 'baseball player', 'driver', 'skier', 'cyclist', 'baby', 'couple', 'umpire', 'crowd', 'gentleman', 'child']
animal
	 ['cat', 'bear', 'chicken', 'bird', 'goat', 'butterfly', 'donkey', 'sheep', 'bull', 'dog', 'giraffe', 'polar bear', 'elephant', 'frog', 'calf', 'puppy', 'dragon', 'kitten', 'cow', 'crab', 'duck', 'horse', 'zebra', 'seagull', 'lamb']
furniture
	 ['bookshelf', 'dresser', 'desk', 'computer desk', 'armchair', 'cupboard', 'bed', 'coffee table', 'nightstand', 'table', 'chair', 'couch', 'bar stool', 'shelf', 'dining table', 'tv stand', 'cabinet', 'bookcase', 'end table', 'ottoman', 'office chair', 'sofa', 'side table', 'closet', 'drawer']
device
	 ['computer mouse', 'keyboard', 'earphone', 'printer', 'hair dryer', 'cell phone', 'laptop', 'remote

In [25]:
t5_pred=load_pkl('../gqa_t5_pred/t5_generated_temp.pkl')

In [26]:
def ans_open_opt_candi(bbox,input_image,ques_id,obj,showing=False):
    #prompt_head='it is a photo of '
    if obj not in valid_ans_cat:
        #candidates=other_ans_list
        #candidates=bart_dict[ques_id]
        valid_items=t5_pred[ques_id]
        #print (valid_items)
    else:
        valid_items=[o for o in valid_ans_cat[obj]]
    #print (valid_items[:50])
    sents=[o for o in valid_items]
    #print (sents)
    text = clip.tokenize(sents).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text)
        img_prepro=get_cropped(bbox,input_image)
        #display.display(img_prepro)
        image_features = clip_model.encode_image(preprocess(img_prepro).unsqueeze(0).to(device))
        
    text_features /= text_features.norm(dim=-1, keepdim=True)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (values,indices)
    return valid_items[indices[0][0].item()],values[0][0].item()

In [27]:
def ans_open_query_candi(bbox,input_image,ques_id,showing=False):
    #prompt_head='it is a photo of '
    #candis=bart_dict[ques_id]
    #candis=load_json(os.path.join(GQA_PATH,'features',ques_id+'.json'))
    candis=t5_pred[ques_id]
    #print (candis)
    valid_items=[o for o in candis]
    #print (valid_items[:50])
    sents=[o for o in valid_items]
    #print (sents)
    text = clip.tokenize(sents).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text)
        img_prepro=get_cropped(bbox,input_image)
        image_features = clip_model.encode_image(preprocess(img_prepro).unsqueeze(0).to(device))
        
    text_features /= text_features.norm(dim=-1, keepdim=True)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(5)
    #print (values,indices)
    return valid_items[indices[0][0].item()],values[0][0].item()

In [28]:
def answer_query(layout,img_id,ques_id,showing=False,texts=None):
    content=layout[-1]['argument'][0]
    if content=='hposition':
        pred=ans_query_hposition(layout,img_id,ques_id,showing,texts)
    elif content in ['place','color','material']:
        candi_dict={'place':scene_list,'color':color_list,'material':material_list}
        pred,_=answer_candi(img_id,candi_dict[content],content,ques_id,layout,showing)
        if showing:
            texts+='\nPrediction:'+pred
            show_img_with_text(os.path.join(GQA_PATH,'images',img_id+'.jpg'),texts)     
    elif content in ['size','length','height']:
        pred=ans_coord_rela(layout,img_id,ques_id,content,showing,texts)
    else:
        last_second_op=layout[-2]['operation']
        if last_second_op=='relocate':
            pred=generate_rela_query(layout,img_id,ques_id,showing,texts)
        else:
            pred=generate_basic_query(layout,img_id,ques_id,showing,texts)
    return pred

In [29]:
def generate_basic_query(layout,img_id,ques_id,showing=False,texts=None):
    cap,_=cap_generator(layout,len(layout)-2,img_id)
    obj=layout[-2]['argument'][0]
    if obj=='scene':
        bbox=[0.5,0.5,0.99,0.99]
        score=0.9
    else:
        score,bbox=generate_phrase_ground(img_id,cap,obj)
    input_image=get_gqa_feat(img_id)
    pred,_=ans_open_opt_candi(bbox,input_image,ques_id,obj,showing)
    if showing:
        texts+='\nCaption:'+cap
        texts+='\nPrediction:'+pred
        vis_bbox_only([bbox],[score],input_image,text=texts)
    return pred

In [30]:
def answer_candi(img_id,candidates,content,ques_id,layout,showing=False):
    sents=[p for p in candidates]
    text = clip.tokenize(sents).to(device)
    img_path=os.path.join(GQA_PATH,'images',img_id+'.jpg')
    with torch.no_grad():
        text_features = clip_model.encode_text(text)
        if content=='place':
            image_features = clip_model.encode_image(preprocess(Image.open(img_path)).unsqueeze(0).to(device))
        else:
            if len(layout)==2:
                score,bbox,_=bbox_generator(layout,0,img_id,ques_id)
            else:
                cap,_=cap_generator(layout,len(layout)-2,img_id)
                obj=layout[0]['argument'][0]
                score,bbox=generate_phrase_ground(img_id,cap,obj)
            input_image=get_gqa_feat(img_id)
            if showing:
                vis_bbox_only([bbox],[score],input_image)
            img_prepro=get_cropped(bbox,input_image)
            image_features = clip_model.encode_image(preprocess(img_prepro).unsqueeze(0).to(device))
    text_features /= text_features.norm(dim=-1, keepdim=True)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(5)
    #print (values,indices)
    return candidates[indices[0][0].item()],values[0][0].item()

In [31]:
def ans_coord_rela(layout,img_id,ques_id,content,showing=False,texts=None):
    if len(layout)==2:
        score,bbox,_=bbox_generator(layout,0,img_id,ques_id)
    else:
        cap,_=cap_generator(layout,len(layout)-2,img_id)
        obj=layout[0]['argument'][0]
        score,bbox=generate_phrase_ground(img_id,cap,obj)
    if content=='size':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='large'
        else:
            pred='small'
    elif content=='length':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='long'
        else:
            pred='short' 
    elif content=='height':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='tall'
        else:
            pred='short' 
    if showing:
        input_image=get_gqa_feat(img_id)
        texts+='\n'+str(bbox[2])+','+str(bbox[3])
        texts+='\nPrediction:'+pred
        vis_bbox_only([bbox],[score],input_image,text=texts)
    return pred

In [32]:
def ans_coord_rela(layout,img_id,ques_id,content,showing=False,texts=None):
    if len(layout)==2:
        score,bbox,_=bbox_generator(layout,0,img_id,ques_id)
    else:
        cap,_=cap_generator(layout,len(layout)-2,img_id)
        obj=layout[0]['argument'][0]
        score,bbox=generate_phrase_ground(img_id,cap,obj)
    if content=='size':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='large'
        else:
            pred='small'
    elif content=='length':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='long'
        else:
            pred='short' 
    elif content=='height':
        if bbox[2]>0.3 or bbox[3]>0.3:
            pred='tall'
        else:
            pred='short' 
    if showing:
        input_image=get_gqa_feat(img_id)
        texts+='\n'+str(bbox[2])+','+str(bbox[3])
        texts+='\nPrediction:'+pred
        vis_bbox_only([bbox],[score],input_image,text=texts)
    return pred

In [33]:
def ans_query_hposition(layout,img_id,ques_id,showing=False,texts=None):
    if len(layout)==2:
        score,bbox,_=bbox_generator(layout,0,img_id,ques_id)
    else:
        cap,_=cap_generator(layout,len(layout)-2,img_id)
        obj=layout[0]['argument'][0]
        score,bbox=generate_phrase_ground(img_id,cap,obj)
    if bbox[0]<0.5:
        pred='left'
    else:
        pred='right'
    if showing:
        input_image=get_gqa_feat(img_id)
        texts+='\nPrediction:'+pred
        vis_bbox_only([bbox],[score],input_image,text=texts)
    return pred

In [34]:
meta_prog=load_pkl('../meta_generated_layout/dep_all_meta_layout.pkl')

In [36]:
#BART Help
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
showing=False
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]
    if row['types']['structural']!='query':
        continue
    texts=[' '.join([str(vis),row['question'],row['answer'],row['imageId']])]
    for step in new_prog:
        texts.append('--'.join([step['operation'],step['argument'][0]]))
    texts='\n'.join(texts)
    #pred=answer_query(new_prog,row['imageId'],name,
    #                      showing=showing,texts=texts)
    #texts.append('\tPrediction:'+pred)
    #texts='\n'.join(texts)
    #print (texts,'\n')
    
    try:
        pred=answer_query(new_prog,row['imageId'],name,
                          showing=showing,texts=texts)
    except:
        invalid.append(name)
    #print (texts,'\nPrediction:',pred,'\n')
    if pred==row['answer']:
        acc+=1
    vis+=1
    
    if vis%500==0:
        print ('already finished:',vis)
        print ('\tAccuracy:',acc/vis*100.0)
        print (len(invalid))
print (vis)
print (len(invalid))
print (acc/vis*100.0)

11
0
45.45454545454545


In [37]:
#BART Help
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
showing=False
acc=0.0
invalid=[]
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    new_prog=meta_prog[name]
    if row['types']['structural']!='query':
        continue
    texts=[' '.join([str(vis),row['question'],row['answer'],row['imageId']])]
    for step in new_prog:
        texts.append('--'.join([step['operation'],step['argument'][0]]))
    texts='\n'.join(texts)
    #pred=answer_query(new_prog,row['imageId'],name,
    #                      showing=showing,texts=texts)
    #texts.append('\tPrediction:'+pred)
    #texts='\n'.join(texts)
    #print (texts,'\n')
    
    try:
        pred=answer_query(new_prog,row['imageId'],name,
                          showing=showing,texts=texts)
    except:
        invalid.append(name)
    #print (texts,'\nPrediction:',pred,'\n')
    if pred==row['answer']:
        acc+=1
    vis+=1
    total[name]=pred
    if vis%100==0:
        print ('already finished:',vis)
        print ('\tAccuracy:',acc/vis*100.0)
        print (len(invalid))
print (vis)
print (len(invalid))
print (acc/vis*100.0)

Invalid caption generation for 2412014 person painting on person in front of wall 
already finished: 100
	Accuracy: 23.0
1
already finished: 200
	Accuracy: 23.0
1
Invalid caption generation for 2412014 place table in place 
already finished: 300
	Accuracy: 22.333333333333332
2
already finished: 400
	Accuracy: 25.25
2
already finished: 500
	Accuracy: 25.6
2
already finished: 600
	Accuracy: 24.166666666666668
2
already finished: 700
	Accuracy: 24.142857142857142
2
already finished: 800
	Accuracy: 24.0
2
already finished: 900
	Accuracy: 24.666666666666668
2
Invalid caption generation for 2334149 vehicle building behind vehicle 
already finished: 1000
	Accuracy: 25.1
3
already finished: 1100
	Accuracy: 24.454545454545453
3
already finished: 1200
	Accuracy: 24.5
3
already finished: 1300
	Accuracy: 25.0
3
already finished: 1400
	Accuracy: 25.357142857142854
3
already finished: 1500
	Accuracy: 25.6
3
already finished: 1600
	Accuracy: 25.75
3
already finished: 1700
	Accuracy: 26.17647058823529

already finished: 13500
	Accuracy: 25.792592592592595
16
already finished: 13600
	Accuracy: 25.77205882352941
16
already finished: 13700
	Accuracy: 25.824817518248178
16
already finished: 13800
	Accuracy: 25.855072463768114
16
already finished: 13900
	Accuracy: 25.81294964028777
16
already finished: 14000
	Accuracy: 25.814285714285713
16
Invalid caption generation for 2324040 item item on wall 
already finished: 14100
	Accuracy: 25.886524822695034
17
already finished: 14200
	Accuracy: 25.943661971830984
17
already finished: 14300
	Accuracy: 25.930069930069934
17
already finished: 14400
	Accuracy: 25.881944444444443
17
already finished: 14500
	Accuracy: 25.90344827586207
17
already finished: 14600
	Accuracy: 25.945205479452056
17
already finished: 14700
	Accuracy: 25.911564625850342
17
already finished: 14800
	Accuracy: 25.91216216216216
17
already finished: 14900
	Accuracy: 25.919463087248324
17
already finished: 15000
	Accuracy: 25.926666666666666
17
already finished: 15100
	Accuracy:

already finished: 26400
	Accuracy: 25.96969696969697
30
already finished: 26500
	Accuracy: 25.958490566037735
30
already finished: 26600
	Accuracy: 25.977443609022554
30
already finished: 26700
	Accuracy: 25.985018726591758
30
Invalid caption generation for 2363979 bed pillow lying on bed 
already finished: 26800
	Accuracy: 25.988805970149254
30
Invalid caption generation for 2334149 vehicle vehicle on road 
already finished: 26900
	Accuracy: 25.98884758364312
31
already finished: 27000
	Accuracy: 25.992592592592594
31
Invalid caption generation for 2324040 person person wearing suit 
already finished: 27100
	Accuracy: 25.977859778597782
32
Invalid caption generation for 2412014 item item in front of men behind painting 
already finished: 27200
	Accuracy: 25.981617647058826
33
already finished: 27300
	Accuracy: 26.01831501831502
33
already finished: 27400
	Accuracy: 26.032846715328468
33
already finished: 27500
	Accuracy: 26.007272727272728
33
already finished: 27600
	Accuracy: 26.0144

already finished: 38500
	Accuracy: 26.215584415584413
48
already finished: 38600
	Accuracy: 26.19689119170984
48
already finished: 38700
	Accuracy: 26.196382428940566
48
Invalid caption generation for 2324040 item item on left shelf 
already finished: 38800
	Accuracy: 26.188144329896907
49
already finished: 38900
	Accuracy: 26.17737789203085
49
already finished: 39000
	Accuracy: 26.166666666666664
49
already finished: 39100
	Accuracy: 26.14833759590793
49
already finished: 39200
	Accuracy: 26.14030612244898
49
already finished: 39300
	Accuracy: 26.139949109414758
49
already finished: 39400
	Accuracy: 26.121827411167516
49
already finished: 39500
	Accuracy: 26.131645569620254
49
already finished: 39600
	Accuracy: 26.1489898989899
49
already finished: 39700
	Accuracy: 26.1360201511335
49
Invalid caption generation for 2391784 item man to the left of horse holding item 
already finished: 39800
	Accuracy: 26.14321608040201
50
already finished: 39900
	Accuracy: 26.13533834586466
50
already 

already finished: 50600
	Accuracy: 26.369565217391305
67
already finished: 50700
	Accuracy: 26.384615384615383
67
already finished: 50800
	Accuracy: 26.381889763779526
67
already finished: 50900
	Accuracy: 26.38310412573674
67
already finished: 51000
	Accuracy: 26.388235294117646
67
already finished: 51100
	Accuracy: 26.403131115459882
67
already finished: 51200
	Accuracy: 26.41015625
67
already finished: 51300
	Accuracy: 26.405458089668617
67
already finished: 51400
	Accuracy: 26.394941634241242
67
already finished: 51500
	Accuracy: 26.407766990291265
67
Invalid caption generation for 2329391 person person wearing cap 
already finished: 51600
	Accuracy: 26.41279069767442
68
already finished: 51700
	Accuracy: 26.41972920696325
68
already finished: 51800
	Accuracy: 26.43822393822394
68
already finished: 51900
	Accuracy: 26.452793834296724
68
already finished: 52000
	Accuracy: 26.43076923076923
68
already finished: 52100
	Accuracy: 26.426103646833017
68
Invalid caption generation for 233

already finished: 63100
	Accuracy: 26.380348652931858
83
Invalid caption generation for 2325221 item person to the right of people wearing item 
already finished: 63200
	Accuracy: 26.3876582278481
84
Invalid caption generation for 2324329 fire fire 
already finished: 63300
	Accuracy: 26.40126382306477
84
already finished: 63400
	Accuracy: 26.4006309148265
84
already finished: 63500
	Accuracy: 26.40472440944882
84
already finished: 63600
	Accuracy: 26.416666666666664
84
Invalid caption generation for 2325221 not gray not gray clothing 
already finished: 63700
	Accuracy: 26.42072213500785
85
already finished: 63800
	Accuracy: 26.43573667711599
85
already finished: 63900
	Accuracy: 26.425665101721442
85
already finished: 64000
	Accuracy: 26.437500000000004
85
already finished: 64100
	Accuracy: 26.446177847113884
85
already finished: 64200
	Accuracy: 26.450155763239874
85
already finished: 64300
	Accuracy: 26.455676516329707
85
already finished: 64400
	Accuracy: 26.459627329192546
85
alrea

In [38]:
pkl.dump(total,open('query.pkl','wb'))

In [50]:
#BART Help
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
showing=False
acc=0.0
invalid=[]
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]
    if row['types']['structural']!='query':
        continue
    texts=[' '.join([str(vis),row['question'],row['answer'],row['imageId']])]
    for step in new_prog:
        texts.append('--'.join([step['operation'],step['argument'][0]]))
    texts='\n'.join(texts)
    #pred=answer_query(new_prog,row['imageId'],name,
    #                      showing=showing,texts=texts)
    #texts.append('\tPrediction:'+pred)
    #texts='\n'.join(texts)
    #print (texts,'\n')
    
    try:
        pred=answer_query(new_prog,row['imageId'],name,
                          showing=showing,texts=texts)
    except:
        invalid.append(name)
    #print (texts,'\nPrediction:',pred,'\n')
    if pred==row['answer']:
        acc+=1
    vis+=1
    
    if vis%500==0:
        print ('already finished:',vis)
        print ('\tAccuracy:',acc/vis*100.0)
        print (len(invalid))
print (vis)
print (len(invalid))
print (acc/vis*100.0)

Invalid caption generation for 2334149 vehicle vehicle in front of building 
already finished: 500
	Accuracy: 26.200000000000003
1
Invalid caption generation for 2412014 furniture furniture in office 
already finished: 1000
	Accuracy: 28.1
2
already finished: 1500
	Accuracy: 28.199999999999996
2
already finished: 2000
	Accuracy: 27.500000000000004
2
already finished: 2500
	Accuracy: 27.16
2
Invalid caption generation for 2325221 item person to the right of people wearing item 
Invalid caption generation for 2325221 gray gray clothing 
already finished: 3000
	Accuracy: 27.1
4
already finished: 3500
	Accuracy: 27.142857142857142
4
already finished: 4000
	Accuracy: 26.724999999999998
4
already finished: 4500
	Accuracy: 26.84444444444444
4
already finished: 5000
	Accuracy: 27.0
4
already finished: 5500
	Accuracy: 26.81818181818182
4
already finished: 6000
	Accuracy: 26.73333333333333
5
already finished: 6500
	Accuracy: 26.676923076923075
5
Invalid caption generation for 2391784 person pers

Invalid caption generation for 2324040 furniture books on furniture 
Invalid caption generation for 2334149 vehicle vehicle on road 
already finished: 36500
	Accuracy: 26.26027397260274
62
already finished: 37000
	Accuracy: 26.248648648648647
62
Invalid caption generation for 2325221 gray gray clothing 
already finished: 37500
	Accuracy: 26.269333333333332
63
Invalid caption generation for 2412014 person person holding racket 
already finished: 38000
	Accuracy: 26.260526315789473
64
already finished: 38500
	Accuracy: 26.28051948051948
64
already finished: 39000
	Accuracy: 26.274358974358975
64
already finished: 39500
	Accuracy: 26.22278481012658
64
already finished: 40000
	Accuracy: 26.217499999999998
64
already finished: 40500
	Accuracy: 26.200000000000003
64
Invalid caption generation for 2328203 item surfing person riding on item 
already finished: 41000
	Accuracy: 26.151219512195123
65
Invalid caption generation for 882 item item on wall 
already finished: 41500
	Accuracy: 26.16626

In [49]:
#BART Help
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
showing=False
acc=0.0
invalid=[]
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    new_prog=meta_prog[name]
    if row['types']['structural']!='query':
        continue
    texts=[' '.join([str(vis),row['question'],row['answer'],row['imageId']])]
    for step in new_prog:
        texts.append('--'.join([step['operation'],step['argument'][0]]))
    texts='\n'.join(texts)
    pred=answer_query(new_prog,row['imageId'],name,
                          showing=showing,texts=texts)
    #texts.append('\tPrediction:'+pred)
    #texts='\n'.join(texts)
    #print (texts,'\n')
    
    """try:
        pred=answer_query(new_prog,row['imageId'],name,
                          showing=showing,texts=texts)
    except:
        invalid.append(name)"""
    #print (texts,'\nPrediction:',pred,'\n')
    if pred==row['answer']:
        acc+=1
    vis+=1
    
    if vis%500==0:
        print ('already finished:',vis)
        print ('\tAccuracy:',acc/vis*100.0)
        print (len(invalid))
print (vis)
print (len(invalid))
print (acc/vis*100.0)

11
0
36.36363636363637
